In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec  8 00:26:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   31C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate>0.20.2
!pip install tensorboardX
!pip install transformers[torch]
!pip install peft
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4u2kqwzs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-4u2kqwzs
  Resolved https://github.com/huggingface/transformers to commit 0ea42ef0f9f71deba7775ead33afa0e493823d60
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pydantic-2.5.2-py3-none-any.whl.metadata (65 kB)
INFO: pip is looking at multiple versions of fastapi to determine which version is compatible with other requirements. This could take a while.
  Using cached fastapi-0.104.1-py3-none-any.whl.metadata (24 kB)
Using cached pydantic-2.5.2-py3-none-any.whl (381 kB)
Using cached fastapi-0.104.1-py3-none-any.whl (92 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13


In [3]:
!pip install boto3
!pip install sagemaker
!pip install botocore
!pip install s3fs

  Using cached fastapi-0.95.2-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
Using cached fastapi-0.95.2-py3-none-any.whl (56 kB)
Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.2
    Uninstalling pydantic-2.5.2:
      Successfully uninstalled pydantic-2.5.2
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.104.1
    Uninstalling fastapi-0.104.1:
      Successfully uninstalled fastapi-0.104.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.8.0 requires pydantic>=2.0, but you have pydantic 1.10.13 which is incompatible.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset
import s3fs

# Define your S3 bucket path
s3_bucket_path = 's3://project-whisper-data/dataset/'

# Use s3fs to list files in the bucket
s3 = s3fs.S3FileSystem()
files_in_bucket = s3.ls(s3_bucket_path)

# Print the list of files
print("Files in the S3 bucket:")
print(files_in_bucket)

#s3.get(s3_bucket_path, './whisper_data_tr/', recursive=True, maxdepth=None)


Files in the S3 bucket:
['project-whisper-data/dataset/dataset_dict.json', 'project-whisper-data/dataset/test', 'project-whisper-data/dataset/train']


In [5]:
from datasets import load_dataset, DatasetDict
common_voice = DatasetDict.load_from_disk("whisper_data_tr/dataset")
s3_bucket_path = 's3://project-whisper-data/dataset/'
#common_voice = DatasetDict.load_from_disk(s3_bucket_path)

In [ ]:
model = "openai/whisper-small"

In [6]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained(model)

from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained(model, language="Turkish", task="transcribe")

from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained(model, language="Turkish", task="transcribe")

In [7]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [8]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [9]:
import evaluate

metric = evaluate.load("wer")

In [10]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [24]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small",load_in_8bit=True,device_map="auto")

In [31]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model,output_embedding_layer_name="proj_out")

TypeError: prepare_model_for_kbit_training() got an unexpected keyword argument 'output_embedding_layer_name'

In [26]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [18]:


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    label_names = ['labels']
)

In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [20]:
processor.save_pretrained(training_args.output_dir)

In [21]:
trainer.train()

IndexError: Invalid key: 27417 is out of bounds for size 0

In [28]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "hi",
    "model_name": "Whisper Small Hi TR",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)